In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [57]:
df_mlm = pd.read_csv('Data/data_MLM_2.csv')

In [3]:
df_mlm.head()

,genres,app_name,tags,specs,price,early_access,id,developer
0,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,Strategy Action Indie Casual Simulation,Single-player,4.99,False,761140.0,Kotoshiro
1,Free to Play Indie RPG Strategy,Ironbound,Free to Play Strategy Indie RPG Card Game Trad...,Single-player Multi-player Online Multi-Player...,Free To Play,False,643980.0,Secret Level SRL
2,Casual Free to Play Indie Simulation Sports,Real Pool 3D - Poolians,Free to Play Simulation Sports Casual Indie Mu...,Single-player Multi-player Online Multi-Player...,Free to Play,False,670290.0,Poolians.com
3,Action Adventure Casual,弹炸人2222,Action Adventure Casual,Single-player,0.99,False,767400.0,彼岸领域
4,NaN,Log Challenge,Action Indie Casual Sports,Single-player Full controller support HTC Vive...,2.99,False,773570.0,NaN


In [4]:
df_mlm_v1 = df_mlm.copy()
df_mlm_v2 = df_mlm.copy()

In [5]:
df_mlm_v2 = df_mlm_v2.drop(['genres', 'tags', 'specs', 'app_name', 'id', 'developer'], axis=1)
df_mlm_v2 = df_mlm_v2.replace(to_replace=np.nan, value=0)

In [33]:
print(len(df_mlm_v2))
print(df_mlm_v2.isnull().sum())
df_mlm_v2.head()

32135
price           0
early_access    0
dtype: int64


,price,early_access
0,4.99,0
1,0,0
2,0,0
3,0.99,0
4,2.99,0


In [8]:
df_mlm_v2['price'] = df_mlm_v2['price'].replace(regex=r'[^0-9.]', value='0')
df_mlm_v2['price'] = df_mlm_v2['price'].str.replace(r'00+|0\.0+', '0', regex=True)
df_mlm_v2['price'] = df_mlm_v2['price'].replace('', 0)

In [14]:
df_mlm_v2['early_access'] = df_mlm_v2['early_access'].astype(int)
df_mlm_v2['price'].astype(float)

0        4.99
1        0.00
2        0.00
3        0.99
4        2.99
         ... 
32130    1.99
32131    4.99
32132    1.99
32133    4.99
32134    4.99
Name: price, Length: 32135, dtype: float64

In [16]:
numericos = df_mlm_v2[['price', 'early_access']].values

In [17]:
numericos

array([['4.99', 0],
       ['0', 0],
       ['0', 0],
       ...,
       ['1.99', 0],
       ['4.99', 0],
       ['4.99', 1]], dtype=object)

-------------------------------------------------

In [19]:
print(len(df_mlm_v1))
print(df_mlm_v1.isnull().sum())

32135
genres          3283
app_name           2
tags             163
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


In [20]:
df_mlm_v1 = df_mlm_v1.replace(to_replace=np.nan, value='No data')

df_mlm_v1.astype(str)

,genres,app_name,tags,specs,price,early_access,id,developer
0,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,Strategy Action Indie Casual Simulation,Single-player,4.99,False,761140.0,Kotoshiro
1,Free to Play Indie RPG Strategy,Ironbound,Free to Play Strategy Indie RPG Card Game Trad...,Single-player Multi-player Online Multi-Player...,Free To Play,False,643980.0,Secret Level SRL
2,Casual Free to Play Indie Simulation Sports,Real Pool 3D - Poolians,Free to Play Simulation Sports Casual Indie Mu...,Single-player Multi-player Online Multi-Player...,Free to Play,False,670290.0,Poolians.com
3,Action Adventure Casual,弹炸人2222,Action Adventure Casual,Single-player,0.99,False,767400.0,彼岸领域
4,No data,Log Challenge,Action Indie Casual Sports,Single-player Full controller support HTC Vive...,2.99,False,773570.0,No data
...,...,...,...,...,...,...,...,...
32130,Casual Indie Simulation Strategy,Colony On Mars,Strategy Indie Casual Simulation,Single-player Steam Achievements,1.99,False,773640.0,"Nikita ""Ghost_RUS"""
32131,Casual Indie Strategy,LOGistICAL: South Africa,Strategy Indie Casual,Single-player Steam Achievements Steam Cloud S...,4.99,False,733530.0,Sacada
32132,Indie Racing Simulation,Russian Roads,Indie Simulation Racing,Single-player Steam Achievements Steam Trading...,1.99,False,610660.0,Laush Dmitriy Sergeevich
32133,Casual Indie,EXIT 2 - Directions,Indie Casual Puzzle Singleplayer Atmospheric R...,Single-player Steam Achievements Steam Cloud,4.99,False,658870.0,"xropi,stev3ns"


In [22]:
print(len(df_mlm_v1))
print(df_mlm_v1.isnull().sum())

32135
genres          0
app_name        0
tags            0
specs           0
price           0
early_access    0
id              0
developer       0
dtype: int64


In [24]:
vectorizer_TFIDF = TfidfVectorizer()

In [25]:
TFIDF_appname = vectorizer_TFIDF.fit_transform(df_mlm_v1['app_name'])
TFIDF_genres = vectorizer_TFIDF.fit_transform(df_mlm_v1['genres'])
TFIDF_tags = vectorizer_TFIDF.fit_transform(df_mlm_v1['tags'])
TFIDF_specs = vectorizer_TFIDF.fit_transform(df_mlm_v1['specs'])
TFIDF_developer = vectorizer_TFIDF.fit_transform(df_mlm_v1['developer'])


In [26]:
TFIDF_appname = TFIDF_appname.toarray()
TFIDF_genres = TFIDF_genres.toarray()
TFIDF_tags = TFIDF_tags.toarray()
TFIDF_specs = TFIDF_specs.toarray()
TFIDF_developer = TFIDF_developer.toarray()

In [27]:
TFIDF_appname

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
TFIDF_matrix = np.hstack((TFIDF_appname, TFIDF_genres, TFIDF_tags, TFIDF_specs, TFIDF_developer))

In [29]:
print(TFIDF_matrix.shape)
TFIDF_matrix

(32135, 31630)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
svd = TruncatedSVD(n_components=5, random_state=42)

In [31]:
TFIDF_matrix_v2 = svd.fit_transform(TFIDF_matrix)

In [32]:
print(TFIDF_matrix_v2.shape)
TFIDF_matrix_v2

(32135, 5)


array([[ 1.20194453, -0.00228764,  0.38532938,  0.25024752,  0.03119349],
       [ 0.57728191, -0.05274437, -0.08857326,  0.56936868,  0.03014025],
       [ 0.55800765,  0.05092211,  0.34052631,  0.13935168,  0.12776026],
       ...,
       [ 0.87460309, -0.02683127,  0.43570678, -0.12410849,  0.00902235],
       [ 0.95843381, -0.07440055,  0.23099646, -0.21677684, -0.59477257],
       [ 0.39952063,  1.74100599, -0.08880865, -0.03539022,  0.03677075]])

In [34]:
TFIDF_matrix_v3 = np.hstack((TFIDF_matrix_v2, numericos))

In [35]:
print(TFIDF_matrix_v3.shape)
TFIDF_matrix_v3

(32135, 7)


array([[1.2019445290617117, -0.0022876379019705637, 0.38532938268790656,
        ..., 0.03119348983439347, '4.99', 0],
       [0.5772819076658129, -0.05274436570371994, -0.08857325695185297,
        ..., 0.030140248940055864, '0', 0],
       [0.5580076498369053, 0.05092210852654091, 0.3405263119926744, ...,
        0.12776026032845708, '0', 0],
       ...,
       [0.8746030923548814, -0.026831265422981813, 0.4357067794078129,
        ..., 0.009022351972730026, '1.99', 0],
       [0.9584338133484752, -0.0744005515664946, 0.2309964638393589, ...,
        -0.5947725671812983, '4.99', 0],
       [0.39952063040199715, 1.741005987056971, -0.0888086500698664, ...,
        0.03677075260996583, '4.99', 1]], dtype=object)

In [36]:
sim_coseno = cosine_similarity(TFIDF_matrix_v3, TFIDF_matrix_v3)

In [37]:
print(sim_coseno.shape)

(32135, 32135)


In [38]:
indices_sc = np.argsort(-sim_coseno, axis=1)[:, 1:6]

In [40]:
indices_sc[32103]

array([32105, 32114, 32015, 32108, 32104], dtype=int64)

In [54]:
np.save('sim_coseno.npy', indices_sc)